### Import Required Packages and Set Options

#### Import Base Libraries

In [1]:
import os
import sys

import numpy as np
import pandas as pd
import multiprocessing as mp

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from functools import partial

#### Put the Main Package Library on the PYTHONPATH

In [2]:
curdir = sys.path[0]
srcdir = os.path.join(os.path.split(curdir)[0], 'rankfm')
sys.path[0] = srcdir
srcdir

'/Users/ericlundquist/Repos/rankfm/rankfm'

#### Dynamically Re-Load all Package Modules on Execution

In [3]:
%load_ext autoreload
%autoreload 2

from rankfm import RankFM
from evaluation import precision_at_k, recall_at_k

#### Set File Path Constants

In [4]:
REPO_ROOT = os.path.split(srcdir)[0]
DATA_ROOT = os.path.join(REPO_ROOT, "data/ml-100k")
print("\n".join([REPO_ROOT, DATA_ROOT]))

/Users/ericlundquist/Repos/rankfm
/Users/ericlundquist/Repos/rankfm/data/ml-100k


### Prepare Example Data

#### Load Users Data

In [5]:
users_df = pd.read_csv(os.path.join(DATA_ROOT, "users.csv"))
users_df['agegroup'] = pd.cut(users_df['age'], [0, 30, 45, 100], right=False, labels=False)
users_df = users_df.drop(['age', 'zip_code'], axis=1)
users_df = pd.get_dummies(users_df, prefix_sep='__', columns=['agegroup', 'gender', 'occupation'])
users_df.mean()

user_id                      472.000000
agegroup__0                    0.433722
agegroup__1                    0.348887
agegroup__2                    0.217391
gender__F                      0.289502
gender__M                      0.710498
occupation__administrator      0.083775
occupation__artist             0.029692
occupation__doctor             0.007423
occupation__educator           0.100742
occupation__engineer           0.071050
occupation__entertainment      0.019088
occupation__executive          0.033934
occupation__healthcare         0.016967
occupation__homemaker          0.007423
occupation__lawyer             0.012725
occupation__librarian          0.054083
occupation__marketing          0.027572
occupation__none               0.009544
occupation__other              0.111347
occupation__programmer         0.069989
occupation__retired            0.014846
occupation__salesman           0.012725
occupation__scientist          0.032874
occupation__student            0.207847


#### Load Items Data

In [6]:
items_df = pd.read_csv(os.path.join(DATA_ROOT, "items.csv"))
item_names = items_df[['item_id', 'item_name']]
item_names.head()

,item_id,item_name
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [7]:
items_df = items_df.drop(['item_name', 'release_date'], axis=1)
items_df.columns = ['item_id'] + ["genre__{}".format(col) for col in items_df.columns[1:]]
items_df.mean()

item_id               841.500000
genre__action           0.149227
genre__adventure        0.080262
genre__animation        0.024970
genre__childrens        0.072533
genre__comedy           0.300238
genre__crime            0.064804
genre__documentary      0.029727
genre__drama            0.431034
genre__fantasy          0.013080
genre__film_noir        0.014269
genre__horror           0.054697
genre__musical          0.033294
genre__mystery          0.036266
genre__romance          0.146849
genre__scifi            0.060048
genre__thriller         0.149227
genre__war              0.042212
genre__western          0.016052
dtype: float64

#### Load Ratings Data

In [8]:
ratings_explicit = pd.read_csv(os.path.join(DATA_ROOT, "ratings.csv"))
ratings_explicit['timestamp'] = pd.to_datetime(ratings_explicit['unix_timestamp'], origin='unix', unit='s')
ratings_explicit['positive_feedback'] = ratings_explicit.groupby('user_id')['rating'].transform(lambda c: np.where(c > c.mean(), 1, 0))
ratings_explicit = ratings_explicit.drop('unix_timestamp', axis=1)
ratings_explicit.mean()

user_id              462.48475
item_id              425.53013
rating                 3.52986
positive_feedback      0.54194
dtype: float64

In [9]:
ratings_explicit.head()

,user_id,item_id,rating,timestamp,positive_feedback
0,196,242,3,1997-12-04 15:55:49,0
1,186,302,3,1998-04-04 19:22:22,0
2,22,377,1,1997-11-07 07:18:36,0
3,244,51,2,1997-11-27 05:02:03,0
4,166,346,1,1998-02-02 05:33:16,0


#### Generate Implicit Feedback Ratings Data

In [10]:
ratings_implicit = ratings_explicit[ratings_explicit.positive_feedback == 1].reset_index(drop=True)
ratings_implicit.head()

,user_id,item_id,rating,timestamp,positive_feedback
0,253,465,5,1998-04-03 18:34:27,1
1,286,1014,5,1997-11-17 15:38:45,1
2,200,222,5,1997-10-05 09:05:40,1
3,224,29,3,1998-02-21 23:40:57,1
4,122,387,5,1997-11-11 17:47:39,1


#### Print Final Matrix Shapes

In [11]:
print(ratings_explicit.user_id.nunique(), ratings_explicit.item_id.nunique())
print(ratings_implicit.user_id.nunique(), ratings_implicit.item_id.nunique())

print(ratings_explicit.shape)
print(ratings_implicit.shape)

943 1682
943 1483
(100000, 5)
(54194, 5)


#### Create (Training, Validation) Interactions for Evaluation

In [12]:
print(ratings_implicit.shape)
ratings_implicit.head()

(54194, 5)


,user_id,item_id,rating,timestamp,positive_feedback
0,253,465,5,1998-04-03 18:34:27,1
1,286,1014,5,1997-11-17 15:38:45,1
2,200,222,5,1997-10-05 09:05:40,1
3,224,29,3,1998-02-21 23:40:57,1
4,122,387,5,1997-11-11 17:47:39,1


In [174]:
test_pct = 0.25
drop_users = (2, 4)

train_mask = (ratings_implicit.index / ratings_implicit.shape[0]) <  (1 - test_pct)
valid_mask = (ratings_implicit.index / ratings_implicit.shape[0]) >= (1 - test_pct)

interactions_train = ratings_implicit[train_mask][['user_id', 'item_id']]
interactions_valid = ratings_implicit[valid_mask][['user_id', 'item_id']]

# drop some users from the training data to test cold-start functionality
interactions_train = interactions_train[~interactions_train.user_id.isin(drop_users)]

train_users = np.sort(interactions_train.user_id.unique())
valid_users = np.sort(interactions_valid.user_id.unique())
cold_start_users = set(valid_users) - set(train_users)

print("train shape: {}".format(interactions_train.shape))
print("valid shape: {}".format(interactions_valid.shape))
print("train users: {}".format(len(train_users)))
print("valid users: {}".format(len(valid_users)))
print("cold-start users: {}".format(cold_start_users))

train shape: (40598, 2)
valid shape: (13548, 2)
train users: 941
valid users: 905
cold-start users: {2, 4}


### Test Out Core Package Functionality

#### Initialize the Model

In [184]:
model = RankFM(factors=10, learning_rate=0.1, regularization=0.01, sigma=0.1)
model.predict(interactions_valid)

AssertionError: you must fit the model prior to generating predictions

#### Fit the Model on the Training Data and Profile Computational Performance

In [185]:
%%prun -l 10 

model.fit(interactions_train, epochs=10, verbose=True)


training epoch: 0
penalized log-likelihood: -15815.78

training epoch: 1
penalized log-likelihood: -14433.15

training epoch: 2
penalized log-likelihood: -14010.93

training epoch: 3
penalized log-likelihood: -12514.83

training epoch: 4
penalized log-likelihood: -11202.71

training epoch: 5
penalized log-likelihood: -10257.07

training epoch: 6
penalized log-likelihood: -9463.82

training epoch: 7
penalized log-likelihood: -8822.16

training epoch: 8
penalized log-likelihood: -8166.99

training epoch: 9
penalized log-likelihood: -7904.74
 

         4383168 function calls (4381912 primitive calls) in 29.829 seconds

   Ordered by: internal time
   List reduced from 634 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1328   14.720    0.011   14.720    0.011 {method 'acquire' of '_thread.lock' objects}
   405980    9.065    0.000   10.827    0.000 rankfm.py:144(_gradient_step)
   811960    1.929    0.000    3.281    0.000 rankfm.py:132(_pairwise_utility)
   405990    1.534    0.000    3.053    0.000 rankfm.py:181(<genexpr>)
   812020    0.905    0.000    0.919    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
   811960    0.366    0.000    1.352    0.000 <__array_function__ internals>:2(dot)
        1    0.313    0.313   29.825   29.825 rankfm.py:191(fit)
       10    0.164    0.016    0.167    0.017 pool.py:375(_map_async)
      360    0.153    0.000    0.153    0.000 {built-in method posix.waitpid}
       80    0.121    0.002    0.1

#### Evaluate Model Performance on the Validation Data

In [186]:
interactions_valid.head()

,user_id,item_id
40646,437,443
40647,712,1037
40648,903,1070
40649,151,629
40650,711,79


In [187]:
k = 10

In [188]:
# %%prun -l 10 

model_pre = round(precision_at_k(model, interactions_valid, k=k), 3)
model_rec = round(recall_at_k(model, interactions_valid, k=k), 3)
print("model precision: {} model recall: {}".format(model_pre, model_rec))

model precision: 0.121 model recall: 0.129


#### Evaluate Pure Popularity Baseline

In [189]:
most_popular = interactions_train.groupby('item_id').size().sort_values(ascending=False)[:k]
most_popular

item_id
50     357
100    291
181    277
258    260
98     249
174    246
127    245
1      225
56     223
313    221
dtype: int64

In [190]:
test_user_items = interactions_valid.groupby('user_id')['item_id'].apply(set).to_dict()
test_recommends = model.recommend_for_users(list(test_user_items.keys()), cold_start='drop')
test_user_items = {key: val for key, val in test_user_items.items() if key in test_recommends.index}

base_pre = round(np.mean([len(set(most_popular.index) & set(val)) / len(set(most_popular.index)) for key, val in test_user_items.items()]), 3)
base_rec = round(np.mean([len(set(most_popular.index) & set(val)) / len(set(val))                for key, val in test_user_items.items()]), 3)

print("number of test users: {}".format(len(test_user_items)))
print("baseline precision: {} baseline recall: {}".format(base_pre, base_rec))

number of test users: 903
baseline precision: 0.098 baseline recall: 0.088


#### Generate Model Scores for Test User/Items

In [191]:
mask = (interactions_valid.user_id.isin(interactions_train.user_id)) & (interactions_valid.item_id.isin(interactions_train.item_id))
print(interactions_valid.shape)
print(interactions_valid[mask].shape)

(13548, 2)
(13469, 2)


In [192]:
scores_miss = model.predict(interactions_valid, cold_start='nan')
scores_drop = model.predict(interactions_valid, cold_start='drop')

print(scores_miss.shape)
print(scores_drop.shape)

(13548,)
(13469,)


In [193]:
scores_miss[np.isnan(scores_miss)]

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan])

In [194]:
print(scores_miss.mean(), scores_miss.std(), scores_miss.min(), scores_miss.max())
print(scores_drop.mean(), scores_drop.std(), scores_drop.min(), scores_drop.max())

nan nan nan nan
1.479448245323385 1.028846315563963 -3.1809892579649235 5.0855047452400175


#### Spot-Check Some User Recommendations

In [195]:
len(train_users), len(valid_users), cold_start_users

(941, 905, {2, 4})

In [196]:
recommendations = model.recommend_for_users(test_users, n_items=10, cold_start='nan')
print(recommendations.shape)
recommendations.head(10)

(905, 10)


,0,1,2,3,4,5,6,7,8,9
1,174.0,475.0,175.0,50.0,12.0,408.0,186.0,144.0,56.0,42.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,690.0,328.0,313.0,333.0,347.0,270.0,258.0,748.0,292.0,750.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,174.0,403.0,186.0,68.0,173.0,22.0,181.0,50.0,96.0,4.0
6,134.0,514.0,191.0,483.0,705.0,478.0,23.0,647.0,661.0,318.0
7,174.0,183.0,234.0,318.0,22.0,132.0,480.0,143.0,429.0,185.0
8,174.0,22.0,181.0,318.0,50.0,258.0,173.0,172.0,96.0,210.0
9,258.0,313.0,50.0,7.0,269.0,181.0,100.0,302.0,288.0,286.0
10,191.0,318.0,134.0,174.0,483.0,514.0,132.0,603.0,23.0,357.0


In [197]:
recommendations[recommendations.isnull().any(axis=1)]

,0,1,2,3,4,5,6,7,8,9
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [198]:
user_id = 6
user_recs = recommendations.loc[user_id]
user_item_names = item_names[item_names.item_id.isin(user_recs)].set_index('item_id').loc[user_recs]
user_item_names

,item_name
item_id,
134.0,Citizen Kane (1941)
514.0,Annie Hall (1977)
191.0,Amadeus (1984)
483.0,Casablanca (1942)
705.0,Singin' in the Rain (1952)
478.0,"Philadelphia Story, The (1940)"
23.0,Taxi Driver (1976)
647.0,Ran (1985)
661.0,High Noon (1952)


### Look at the Movies with the Highest/Lowest Model Weights

In [91]:
item_weights = pd.DataFrame({'item_idx': np.arange(len(model.w_item)), 'w_item': model.w_item})
item_weights['item_id'] = item_weights['item_idx'].map(model.index_to_item)
item_weights = pd.merge(item_weights, item_names, on='item_id').sort_values('w_item', ascending=False)
item_weights = item_weights[['item_id', 'item_idx', 'item_name', 'w_item']]

#### Best Movies Ever?

In [92]:
item_weights.head(10)

,item_id,item_idx,item_name,w_item
171,174,171,Raiders of the Lost Ark (1981),2.455825
49,50,49,Star Wars (1977),2.426783
6,7,6,Twelve Monkeys (1995),2.391095
0,1,0,Toy Story (1995),2.331772
99,100,99,Fargo (1996),2.308501
178,181,178,Return of the Jedi (1983),2.279695
310,313,310,Titanic (1997),2.265653
299,302,299,L.A. Confidential (1997),2.191196
124,127,124,"Godfather, The (1972)",2.128488
97,98,97,"Silence of the Lambs, The (1991)",2.113763


#### Worst Movies Ever?

In [44]:
item_weights.tail(10)

,item_id,item_idx,item_name,w_item
1349,1498,1349,Farmer & Chase (1995),-1.604219
1360,1515,1360,Wings of Courage (1995),-1.604629
1099,1146,1099,Calendar Girl (1993),-1.605805
1383,1554,1383,Safe Passage (1994),-1.606098
1272,1385,1272,Roseanna's Grave (For Roseanna) (1997),-1.606560
1347,1492,1347,Window to Paris (1994),-1.608512
1345,1490,1345,Fausto (1993),-1.615045
1036,1076,1036,"Pagemaster, The (1994)",-1.623692
1379,1543,1379,Johns (1996),-1.647581
1221,1290,1221,Country Life (1994),-1.659661


# Start Sandbox Code